In [3]:
import pandas as pd 

import spacy
import neuralcoref

from pathlib import Path

from email import policy
from email.parser import BytesParser
import json
import nltk.data
#for fn in hitlist: #get all the details of this meeting/conf by entity extraction using google api
import re

from re import search
import os
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)
#finding the files that has the keyword like meeting or conference in their text. 
hitlist=[]
user_input = input('What is the name of your directory')
directory = os.listdir(user_input)
#print(directory)
eno=0;
sublist=[]
newdict={}
#newdict = {'Subject': ('fname','date')}
i=1
punctuations = '''!()—-―[]{};:'"\<>/?@#$%^&*_~'''
for fname in directory:
        eno = eno+1
        if os.path.isfile(user_input + os.sep + fname):
        # Full path
        #f = open(user_input + os.sep + fname, 'r')
            #print(fname)
            with open(user_input + os.sep + fname, 'rb') as f:
                name = f.name  # Get file name
                msg = BytesParser(policy=policy.default).parse(f)
                
            #print(msg['Subject'])
                
                subj = msg['Subject']
                date = msg['Date']
                #print(date)
                if (subj == " ") or (subj is None):
                    
                    subj="empty"+str(i)
                    i=i+1
                    #print(subj)
                    newdict[subj]=(fname,date)
                   
                    
                else:
                    #print(subj)
                    subj = subj.replace("Re:", "")
                    subj = subj.replace("RE:", "")
                    subj = subj.replace("FW:", "")
                    subj = subj.replace("FWD:", "")
                    subj = subj.replace("Fw:", "")
                    subj = subj.replace("Fwd:", "")
                    subj = subj.replace(" ", "")
                    subj = re.sub("\s+","",subj)
                    subj = subj.strip()
                    
                    #subject already in list; then move this fname to 
                    #if subj in subdict['subject']:
                     #   subdict['file'] = [fname]
                    #else:
                    no_punct = ""
                    for char in subj:
                        if char not in punctuations:
                            no_punct = no_punct + char
                    
                    subj = no_punct
                    #print(date)
                        
                
                        
                        
                    if subj not in newdict:
                        #print('new subj',subj)
                      
                        newdict[subj]= (fname,date)  
                  
                    elif subj in newdict:
                        if (newdict[subj][1] < date) :
                            
                            
                            #newdict[subj].append(fname)
                            newdict[subj]= (fname,date)  
                            #newdict[subj][1] = [date]
                        
                    #sublist.append(subj)
            f.close()
            
print('Done')  


with open('emailchainsapril.txt', 'w') as f1:
    
    
    print('Filename:', newdict, file=f1)
    travelfilter(newdict)
    
f1.close()

    
            

/Users/aishwaryavijayan/recent-env/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/aishwaryavijayan/recent-env/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for mo

What is the name of your directoryemails-podesta
Done
in travelfilter


KeyboardInterrupt: 

In [24]:
def travelfilter(dict):

    keywords=['travel','landed','trip','flying','visited','visit','journeyed','en route','meeting']
    enum=0
    for key in dict:
        
            fname=dict[key][0]
            enum=enum+1
            #print(fname[l-1])
            
            if os.path.isfile(user_input + os.sep + fname):
        # Full path
        #f = open(user_input + os.sep + fname, 'r')
                #print(fname)
                with open(user_input + os.sep + fname, 'rb') as f:
                    name = f.name  # Get file name
                    msg = BytesParser(policy=policy.default).parse(f)
            
                print(name)
        
                if(msg.get_body(preferencelist=('plain'))):
                    text = msg.get_body(preferencelist=('plain')).get_content()    
                #print(text)
                else:
                    print('invalid')
                #break
                
                #mergedtext += text
                #printx
                
                text = textcleaning(text)
            
            #split to sentence and find if the keyword is in the sentence; perform NER and add this file to hitlist only if 
            #there are a person entity and a location in that sentence
                tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    
                sent= text.split('.')
            
                for t in sent:
                    per=0
                    dat=0
                    locn=0
                    doc=nlp(t)
                    #print('checking')
                    for ent in doc.ents:
        #print(ent.text,ent.label_,ent.start_char,ent.end_char)
                        if ent.label_=="PERSON" or ent.label_=="NORP":
                            per=per+1
                            #perslist.append(ent.text)
                        elif ent.label_=="LOC" or ent.label_== "GPE":
                            locn=locn+1
                            #locnlist.append(ent.text)
                        elif ent.label_=="DATE" :
                            dat=dat+1
            
                for searchstring in keywords:
                    res = bytes(searchstring, 'utf-8')
                    if ( (per > 0) and (locn > 0) and (res.decode("utf-8") in text)):
                #if res.decode("utf-8") in text :
                        print('found string in file',res,fname)
                        if fname[l-1] not in hitlist:
                            hitlist.append(fname)
                        #break
            #f.close()
                #l=l-1
            if enum==1000:
                break;
    print('Done with finding')
    #print(hitlist)
    with open(r'hitlistfile.txt', 'w') as fp:
        for item in hitlist:
        # write each item on a new line
            fp.write("%s\n" % item)
        #print('Done')
    fp.close()
    

In [18]:
 dict_to_return = newdict.copy()

In [21]:
def textcleaning(text):
    ###tokenising the email text
# define punctuation
    punctuations = '''!()—-―[]{};:'"\<>/?@#$%^&*_~'''


# Replace all occurrences of character s with an empty string

    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE)
    text = text.replace("httpgetwrap"," ")
    text= text.replace(" Sec. ", "Secretary")
    text= text.replace(" Rep. ", "Representative")
    text= text.replace(" Sen. ", "Senator")
    text= text.replace(" U.S ", "US")
    text = re.sub('\s+',' ',text)
#text = text.replace("today",msg['Date'])
#print(text)
# remove punctuation from the string
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [ ]:
t= 'Wed, 25 Apr 2012 10:18:42'
s= 'Wed, 25 Apr 2012 15:56:49'


In [25]:
travelfilter(dict_to_return)

emails-podesta/19296.eml
emails-podesta/36275.eml
emails-podesta/39146.eml
emails-podesta/21709.eml
emails-podesta/26066.eml
emails-podesta/5924.eml
emails-podesta/29355.eml
emails-podesta/56581.eml
emails-podesta/2202.eml
emails-podesta/48967.eml
emails-podesta/30604.eml
emails-podesta/2093.eml
emails-podesta/46792.eml
emails-podesta/52677.eml
emails-podesta/11812.eml
emails-podesta/20417.eml
emails-podesta/38258.eml
emails-podesta/23180.eml
emails-podesta/34462.eml
emails-podesta/50071.eml
emails-podesta/55088.eml
emails-podesta/32013.eml
emails-podesta/27329.eml
emails-podesta/22200.eml
emails-podesta/56444.eml
emails-podesta/11906.eml
emails-podesta/37830.eml
emails-podesta/49965.eml
emails-podesta/9878.eml
emails-podesta/32775.eml
emails-podesta/25209.eml
emails-podesta/1117.eml
emails-podesta/42292.eml
emails-podesta/35159.eml
found string in file b'meeting' 35159.eml
emails-podesta/24117.eml
emails-podesta/23678.eml
emails-podesta/52081.eml
emails-podesta/53949.eml
emails-podest

emails-podesta/43615.eml
invalid
emails-podesta/56428.eml
emails-podesta/35783.eml
emails-podesta/7114.eml
emails-podesta/24856.eml
emails-podesta/8227.eml
emails-podesta/15453.eml
emails-podesta/56042.eml
emails-podesta/45264.eml
emails-podesta/32197.eml
emails-podesta/21394.eml
emails-podesta/35250.eml
emails-podesta/55077.eml
emails-podesta/961.eml
emails-podesta/11387.eml
emails-podesta/55711.eml
emails-podesta/797.eml
emails-podesta/1203.eml
emails-podesta/33494.eml
emails-podesta/45502.eml
emails-podesta/23687.eml
emails-podesta/38772.eml
emails-podesta/8541.eml
emails-podesta/17764.eml
emails-podesta/53360.eml
emails-podesta/34560.eml
emails-podesta/43173.eml
emails-podesta/9887.eml
emails-podesta/10207.eml
emails-podesta/26130.eml
emails-podesta/21160.eml
emails-podesta/50669.eml
emails-podesta/48026.eml
emails-podesta/47315.eml
emails-podesta/12345.eml
emails-podesta/17522.eml
emails-podesta/51577.eml
emails-podesta/26927.eml
emails-podesta/18611.eml
emails-podesta/5065.eml
em

emails-podesta/38927.eml
emails-podesta/29194.eml
emails-podesta/44456.eml
emails-podesta/50331.eml
emails-podesta/55600.eml
emails-podesta/48071.eml
emails-podesta/56998.eml
emails-podesta/31903.eml
emails-podesta/49660.eml
emails-podesta/38099.eml
emails-podesta/18349.eml
emails-podesta/10845.eml
emails-podesta/2252.eml
emails-podesta/10715.eml
found string in file b'travel' 10715.eml
emails-podesta/56740.eml
emails-podesta/47895.eml
emails-podesta/9449.eml
emails-podesta/42735.eml
emails-podesta/8316.eml
emails-podesta/11226.eml
emails-podesta/14573.eml
found string in file b'visit' 14573.eml
found string in file b'meeting' 14573.eml
emails-podesta/3524.eml
emails-podesta/47140.eml
emails-podesta/44344.eml
emails-podesta/53638.eml
emails-podesta/8019.eml
emails-podesta/12102.eml
invalid
emails-podesta/31557.eml
emails-podesta/13208.eml
emails-podesta/25962.eml
emails-podesta/24357.eml
emails-podesta/52532.eml
emails-podesta/9313.eml
emails-podesta/14567.eml
emails-podesta/36616.eml


emails-podesta/2882.eml
emails-podesta/27754.eml
emails-podesta/21608.eml
emails-podesta/36881.eml
emails-podesta/28467.eml
emails-podesta/30228.eml
emails-podesta/31136.eml
emails-podesta/48495.eml
emails-podesta/17084.eml
emails-podesta/31893.eml
emails-podesta/19254.eml
emails-podesta/10846.eml
emails-podesta/29129.eml
emails-podesta/21443.eml
emails-podesta/11580.eml
emails-podesta/31650.eml
emails-podesta/49933.eml
emails-podesta/44341.eml
emails-podesta/39115.eml
emails-podesta/34989.eml
emails-podesta/40101.eml
emails-podesta/40250.eml
emails-podesta/45380.eml
emails-podesta/34728.eml
emails-podesta/33047.eml
emails-podesta/1434.eml
emails-podesta/13397.eml
emails-podesta/32359.eml
emails-podesta/50086.eml
emails-podesta/25625.eml
emails-podesta/1808.eml
emails-podesta/42866.eml
emails-podesta/24492.eml
emails-podesta/23240.eml
emails-podesta/9298.eml
emails-podesta/350.eml
emails-podesta/7120.eml
emails-podesta/13383.eml
emails-podesta/8186.eml
emails-podesta/42872.eml
emails-p

In [ ]:
s < t

In [ ]:
if subj == " ":
    print(newdict[subj])

In [ ]:
print(len(newdict))

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc=nlp("Last summer, Clinton went to New York")
for ent in doc.ents:
    print(ent.text,ent.label_)

In [ ]:
print(newdict)

In [ ]:
# load json module
import json

# python dictionary with key value pairs
dict = {'Python' : '.py', 'C++' : '.cpp', 'Java' : '.java'}

# create json object from dictionary
json = json.dumps(newdict)

# open file for writing, "w" 
f = open("dict.json","w")

# write json object to file
f.write(json)

# close file
f.close()



In [ ]:
print(len(newdict))

In [ ]:
newdict1 = {}

In [ ]:
newdict1['idea']=('101.eml','11111')


In [ ]:
print(newdict1['idea'][1])

In [ ]:
i=1
subj="empty"+str(i)
print(subj)


In [11]:
dict= dict_to_return
for key in dict:
        
        l = len(dict[key])
        print(dict[key][0])
        fname=dict[key][0]
        print(fname[l-1])

19296.eml
9
36275.eml
6
39146.eml
9
21709.eml
1
26066.eml
6
5924.eml
9
29355.eml
9
56581.eml
6
2202.eml
2
48967.eml
8
30604.eml
0
2093.eml
0
46792.eml
6
52677.eml
2
11812.eml
1
20417.eml
0
38258.eml
8
23180.eml
3
34462.eml
4
50071.eml
0
55088.eml
5
32013.eml
2
27329.eml
7
22200.eml
2
56444.eml
6
11906.eml
1
37830.eml
7
49965.eml
9
9878.eml
8
32775.eml
2
25209.eml
5
1117.eml
1
42292.eml
2
35159.eml
5
24117.eml
4
23678.eml
3
52081.eml
2
53949.eml
3
34304.eml
4
14802.eml
4
20371.eml
0
30162.eml
0
36560.eml
6
41812.eml
1
36488.eml
6
24067.eml
4
47814.eml
7
36513.eml
6
39624.eml
9
31736.eml
1
37619.eml
7
20365.eml
0
54845.eml
4
37132.eml
7
26714.eml
6
18822.eml
8
29427.eml
9
31268.eml
1
6740.eml
7
36507.eml
6
20238.eml
0
2939.eml
9
31088.eml
1
32483.eml
2
39634.eml
9
22572.eml
2
29224.eml
9
9394.eml
3
55195.eml
5
43198.eml
3
32761.eml
2
16578.eml
6
36251.eml
6
52095.eml
2
34310.eml
4
6587.eml
5
43222.eml
3
34476.eml
4
45705.eml
5
24665.eml
4
23891.eml
3
54382.eml
4
22214.eml
2
42752.eml
2
3

1
40650.eml
0
11379.eml
1
54625.eml
4
12670.eml
2
43359.eml
3
44436.eml
4
49631.eml
9
50657.eml
0
45728.eml
5
6746.eml
7
52254.eml
2
9475.eml
4
14201.eml
4
969.eml
6
42047.eml
2
36936.eml
6
2508.eml
5
41900.eml
1
18413.eml
8
25233.eml
5
20061.eml
0
43700.eml
3
29816.eml
9
4179.eml
1
47117.eml
7
48224.eml
8
21657.eml
1
56589.eml
6
3616.eml
6
40678.eml
0
31081.eml
1
32588.eml
2
43371.eml
3
34371.eml
4
53162.eml
3
39669.eml
9
35142.eml
5
38417.eml
8
47496.eml
7
4228.eml
2
10699.eml
0
1001.eml
0
595.eml
9
22943.eml
2
55513.eml
5
13546.eml
3
1767.eml
7
42709.eml
2
15889.eml
5
52085.eml
2
54793.eml
4
45066.eml
5
27882.eml
7
53604.eml
3
8025.eml
0
15651.eml
5
53381.eml
3
41139.eml
1
7316.eml
3
43417.eml
3
25792.eml
5
26168.eml
6
29233.eml
9
20832.eml
0
3170.eml
1
49884.eml
9
22965.eml
2
36088.eml
6
27319.eml
7
48542.eml
8
18375.eml
8
36839.eml
6
13959.eml
3
51013.eml
1
27185.eml
7
33191.eml
3
11423.eml
1
49890.eml
9
3164.eml
1
20826.eml
0
47665.eml
7
26648.eml
6
48556.eml
8
17139.eml
7
43591.

51431.eml
1
5110.eml
1
51402.eml
1
10338.eml
0
27594.eml
7
39972.eml
9
16991.eml
6
29142.eml
9
42318.eml
2
13631.eml
3
32827.eml
2
21947.eml
1
31791.eml
1
23802.eml
3
8434.eml
4
15240.eml
5
35190.eml
5
18230.eml
8
25383.eml
5
29591.eml
9
3549.eml
5
18938.eml
8
18994.eml
8
25366.eml
5
45531.eml
5
28857.eml
8
44723.eml
4
16761.eml
6
19452.eml
9
39772.eml
9
31982.eml
1
40359.eml
0
34720.eml
4
56345.eml
6
10310.eml
0
36399.eml
6
11277.eml
1
51476.eml
1
6431.eml
4
52.eml
2
52123.eml
2
36953.eml
6
14176.eml
4
44741.eml
4
55894.eml
5
15268.eml
5
1647.eml
6
54552.eml
4
18325.eml
8
23902.eml
3
38972.eml
8
32216.eml
2
43748.eml
3
36834.eml
6
45339.eml
5
6357.eml
3
52645.eml
2
43990.eml
3
9064.eml
0
37662.eml
7
35806.eml
5
1438.eml
4
42456.eml
2
26379.eml
6
21873.eml
1
3659.eml
6
51122.eml
1
37485.eml
7
50052.eml
0
16007.eml
6
20710.eml
0
48303.eml
8
5796.eml
7
11768.eml
1
5886.eml
8
21867.eml
1
2125.eml
1
10462.eml
0
14961.eml
4
51358.eml
1
49517.eml
9
2550.eml
5
48609.eml
8
50046.eml
0
4554.eml

5
12301.eml
2
43628.eml
3
12467.eml
2
12520.eml
2
15308.eml
5
972.eml
7
36572.eml
6
6955.eml
9
50461.eml
0
14016.eml
4
4106.eml
1
39621.eml
9
42250.eml
2
2737.eml
7
32484.eml
2
56225.eml
6
47553.eml
7
49305.eml
9
46036.eml
6
5058.eml
0
41981.eml
1
23537.eml
3
28937.eml
8
19532.eml
9
47328.eml
7
3429.eml
4
40447.eml
0
37817.eml
7
20865.eml
0
2904.eml
9
11185.eml
1
55283.eml
5
27437.eml
7
28704.eml
8
16832.eml
6
5291.eml
2
10026.eml
0
7770.eml
7
23651.eml
3
32984.eml
2
9689.eml
6
741.eml
4
33442.eml
3
23889.eml
3
999.eml
9
35033.eml
5
29226.eml
9
51273.eml
1
18637.eml
8
24880.eml
4
52916.eml
2
25546.eml
5
33324.eml
3
50465.eml
0
53166.eml
3
41494.eml
1
49488.eml
9
39252.eml
9
33206.eml
3
28062.eml
8
46963.eml
6
31533.eml
1
24110.eml
4
36763.eml
6
34808.eml
4
42197.eml
2
31125.eml
1
46977.eml
6
16244.eml
6
12539.eml
2
24440.eml
4
16198.eml
6
48782.eml
8
21734.eml
1
41275.eml
1
35741.eml
5
14957.eml
4
15277.eml
5
55388.eml
5
33773.eml
3
32990.eml
2
23645.eml
3
7740.eml
7
755.eml
5
13345.em

0
3173.eml
1
31459.eml
1
54901.eml
4
51326.eml
1
51010.eml
1
18376.eml
8
37395.eml
7
46360.eml
6
2256.eml
2
36935.eml
6
56019.eml
6
17905.eml
7
51776.eml
1
5264.eml
2
50468.eml
0
32016.eml
2
27838.eml
7
20541.eml
0
43372.eml
3
30056.eml
0
8740.eml
7
14195.eml
4
45703.eml
5
23486.eml
3
55510.eml
5
8998.eml
9
1002.eml
0
596.eml
9
13380.eml
3
22607.eml
2
44582.eml
4
32414.eml
2
45844.eml
5
54791.eml
4
25168.eml
5
409.eml
0
54949.eml
4
8938.eml
9
46395.eml
6
20010.eml
0
56186.eml
6
12893.eml
2
2494.eml
4
10210.eml
0
31253.eml
1
29752.eml
9
17864.eml
7
36672.eml
6
41682.eml
1
45412.eml
5
26869.eml
6
39227.eml
9
21468.eml
1
50291.eml
0
26785.eml
6
40082.eml
0
49918.eml
9
29034.eml
9
27019.eml
7
14082.eml
4
19461.eml
9
39794.eml
9
52486.eml
2
34703.eml
4
18542.eml
8
42695.eml
2
15915.eml
5
42957.eml
2
1823.eml
8
53798.eml
3
14709.eml
4
55448.eml
5
52579.eml
2
52654.eml
2
33078.eml
3
42681.eml
2
53954.eml
3
24504.eml
4
32366.eml
2
22175.eml
2
1837.eml
8
54975.eml
4
35409.eml
5
56980.eml
6
4797

...

3
14223.eml
4
8991.eml
9
22949.eml
2
56010.eml
6
10045.eml
0
2502.eml
5
20186.eml
0
19707.eml
9
27831.eml
7
38911.eml
8
26291.eml
6
40114.eml
0
48548.eml
8
46565.eml
6
49656.eml
9
51019.eml
1
56776.eml
6
10723.eml
0
2264.eml
2
15883.eml
5
52510.eml
2
14545.eml
4
54161.eml
4
1779.eml
7
15897.eml
5
9325.eml
3
34681.eml
4
11399.eml
1
25954.eml
5
44366.eml
4
7308.eml
3
54175.eml
4
26285.eml
6
11429.eml
1
19075.eml
9
36096.eml
6
16346.eml
6
46571.eml
6
37388.eml
7
49642.eml
9
2270.eml
2
10051.eml
0
36928.eml
6
41578.eml
1
46217.eml
6
20192.eml
0
27825.eml
7
4167.eml
1
16420.eml
6
50475.eml
0
40666.eml
0
38905.eml
8
12646.eml
2
54613.eml
4
7285.eml
2
44400.eml
4
9443.eml
4
14237.eml
4
6770.eml
7
8985.eml
9
10079.eml
0
29820.eml
9
18425.eml
8
16408.eml
6
48212.eml
8
8775.eml
7
15101.eml
5
33878.eml
3
53154.eml
3
6758.eml
7
55525.eml
5
22975.eml
2
1037.eml
0
42059.eml
2
55243.eml
5
15667.eml
5
53632.eml
3
1989.eml
9
34871.eml
4
12108.eml
2
43421.eml
3
28910.eml
8
38093.eml
8
17070.eml
7
31909.

42892.eml
2
34904.eml
4
54028.eml
4
20959.eml
0
50266.eml
0
3828.eml
8
51178.eml
1
10642.eml
0
398.eml
9
42662.eml
2
9250.eml
2
14424.eml
4
44213.eml
4
12055.eml
2
44575.eml
4
53009.eml
3
34092.eml
4
9536.eml
5
55478.eml
5
24281.eml
4
42104.eml
2
27788.eml
7
46362.eml
6
50500.eml
0
27950.eml
7
19666.eml
9
36685.eml
6
17893.eml
7
26496.eml
6
10130.eml
0
46376.eml
6
5318.eml
3
50514.eml
0
4006.eml
0
19672.eml
9
3769.eml
7
17887.eml
7
26482.eml
6
54772.eml
4
3633.eml
6
44561.eml
4
15048.eml
5
14356.eml
4
24295.eml
4
42676.eml
2
6177.eml
1
14430.eml
4
52465.eml
2
44207.eml
4
22182.eml
2
12041.eml
2
40061.eml
0
31698.eml
1
50272.eml
0
46410.eml
6
37202.eml
7
51187.eml
1
27011.eml
7
41394.eml
1
10865.eml
0
56830.eml
6
53790.eml
3
25606.eml
5
367.eml
6
23277.eml
3
23511.eml
3
23106.eml
3
401.eml
0
25160.eml
5
54799.eml
4
35373.eml
5
19699.eml
9
39549.eml
9
21306.eml
1
26469.eml
6
31115.eml
1
3782.eml
7
51839.eml
1
40934.eml
0
38657.eml
8
17878.eml
7
31101.eml
1
2488.eml
4
28450.eml
8
40920.em

53192.eml
3
25004.eml
5
45928.eml
5
6946.eml
9
17908.eml
7
31071.eml
1
4189.eml
1
51785.eml
1
5297.eml
2
27613.eml
7
31065.eml
1
51949.eml
1
39439.eml
9
51791.eml
1
38727.eml
8
37414.eml
7
5283.eml
2
28534.eml
8
41582.eml
1
27607.eml
7
33672.eml
3
54831.eml
4
23461.eml
3
6952.eml
9
25776.eml
5
22019.eml
2
1783.eml
7
56798.eml
6
4823.eml
8
21510.eml
1
56940.eml
6
3194.eml
1
49860.eml
9
16746.eml
6
45445.eml
5
34653.eml
4
15845.eml
5
38069.eml
8
49690.eml
9
20626.eml
0
56968.eml
6
36044.eml
6
21538.eml
1
39411.eml
9
20140.eml
0
19919.eml
9
10083.eml
0
30353.eml
0
8957.eml
9
559.eml
5
23449.eml
3
32544.eml
2
25038.eml
5
9485.eml
4
12858.eml
2
34121.eml
4
22743.eml
2
11389.eml
1
31059.eml
1
37428.eml
7
20154.eml
0
3816.eml
8
10097.eml
0
49684.eml
9
10929.eml
0
36050.eml
6
16380.eml
6
32236.eml
2
1967.eml
9
42909.eml
2
15689.eml
5
35559.eml
5
34647.eml
4
25992.eml
5
33128.eml
3
24454.eml
4
52529.eml
2
13213.eml
3
7325.eml
3
8016.eml
0
15662.eml
5
48571.eml
8
19058.eml
9
3143.eml
1
20801.eml

25401.eml
5
26108.eml
6
46824.eml
6
39028.eml
9
21667.eml
1
20579.eml
0
34446.eml
4
28125.eml
8
27216.eml
7
16975.eml
6
27570.eml
7
38450.eml
8
37763.eml
7
21101.eml
1
55858.eml
5
23716.eml
3
44955.eml
4
606.eml
0
1392.eml
3
33505.eml
3
13819.eml
3
44994.eml
4
23702.eml
3
55694.eml
5
33511.eml
3
1386.eml
3
612.eml
1
28657.eml
8
18794.eml
8
41839.eml
1
47290.eml
7
3591.eml
5
57083.eml
7
31460.eml
1
40299.eml
0
4598.eml
5
51394.eml
1
11968.eml
1
5686.eml
6
41187.eml
1
35606.eml
5
52845.eml
2
44570.eml
4
12736.eml
2
42101.eml
2
52312.eml
2
29978.eml
9
46367.eml
6
36858.eml
6
17896.eml
7
26493.eml
6
16550.eml
6
27955.eml
7
19663.eml
9
47079.eml
7
36680.eml
6
16236.eml
6
40070.eml
0
31689.eml
1
2300.eml
3
31851.eml
1
6166.eml
1
25824.eml
5
14421.eml
4
12050.eml
2
54005.eml
4
34929.eml
4
7278.eml
2
6172.eml
1
52460.eml
2
9241.eml
2
389.eml
8
54011.eml
4
12044.eml
2
44202.eml
4
22187.eml
2
50277.eml
0
4765.eml
7
20948.eml
0
2314.eml
3
30583.eml
0
56606.eml
6
10653.eml
0
49726.eml
9
51169.eml


4767.eml
7
15729.eml
5
12046.eml
2
32396.eml
2
42671.eml
2
33088.eml
3
25832.eml
5
44957.eml
4
7939.eml
9
1390.eml
3
604.eml
0
34268.eml
4
45491.eml
5
3587.eml
5
2841.eml
8
47286.eml
7
38452.eml
8
2699.eml
6
28641.eml
8
27572.eml
7
30768.eml
0
28127.eml
8
27214.eml
7
37007.eml
7
21665.eml
1
29239.eml
9
31476.eml
1
5848.eml
8
7093.eml
0
176.eml
7
28133.eml
8
38320.eml
8
37013.eml
7
5684.eml
6
21671.eml
1
46832.eml
6
31304.eml
1
26678.eml
6
57081.eml
7
39758.eml
9
21117.eml
1
2855.eml
8
47292.eml
7
19488.eml
9
20209.eml
0
38446.eml
8
28655.eml
8
16963.eml
6
55696.eml
5
610.eml
1
1384.eml
3
23700.eml
3
26650.eml
6
37985.eml
7
29563.eml
9
41813.eml
1
48189.eml
8
39770.eml
9
16793.eml
6
30032.eml
0
638.eml
3
24047.eml
4
7905.eml
9
23728.eml
3
14184.eml
4
34254.eml
4
13833.eml
3
12293.eml
2
8388.eml
3
30754.eml
0
48837.eml
8
11942.eml
1
20547.eml
0
36325.eml
6
21659.eml
1
5874.eml
8
29205.eml
9
10490.eml
0
30740.eml
0
29211.eml
9
32157.eml
2
9082.eml
0
43976.eml
3
33249.eml
3
24053.eml
4
224